## Imports

In [52]:
# Import Statements

# Sleeper wrapper
from sleeper_wrapper import League, Stats

# Progress Bar
from tqdm import tqdm

# Misc
import requests, pandas as pd, numpy as np, warnings as warnings
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')

## Globals

In [2]:
# Find your league
league = League(728718179756519424)

# Create look up dictionary for roster IDs
lookup_dict = {1: 'Robby', 2: 'Matt', 3: 'Tony', 4: 'Tim', 5: 'Kade', 6: 'BJ', 7: 'Erik', 8: 'Joey', 9: 'Ric', 10: 'Chris'}

# Create roster requirements dictionary
requirements_dict = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DEF': 1, 'FLEX': 2}

# Which week to run program for
week = None

## Helper Functions

In [48]:
# Function to parse the players json and turn it into a dataframe
def parse_players(players_json):
    # Create lists for data that we care about
    last_names = []
    first_names = []
    positions = []
    ids = []

    # Loop through the dictionary
    for k,v in tqdm(players_json.items()):
        # Create a temp dict
        item = players_json[k]
        # Save the metadata we carry about
        last_names.append(item['last_name'])
        first_names.append(item['first_name'])
        positions.append(item['position'])
        ids.append(item['player_id'])
    
    # Create a list of full/whole names
    names = ["{} {}".format(x,y) for x,y in zip(first_names, last_names)]

    # Convert to dataframe
    res_df = pd.DataFrame({'Name': names, 'First Name': first_names, 'Last Name': last_names, 'Position': positions, 'ID': ids})
    
    # Return it
    return res_df

# Get the stats for a list of player IDs
def get_stats(players: list, year: int, week: int) -> list:
    # Create stats object
    stats = Stats()
    # Get weekly stats
    week_stats = stats.get_week_stats("regular", year, week)
    # Return scores for each player
    scores = [stats.get_player_week_score(week_stats, player) for player in players]
    
    return scores    

# Function to retrieve data about the league
def get_league_data(df, players_df):

    # Drop rows where user did not set player
    # df.drop(df.loc[df['starters']=='0'].index, inplace=True)
    df = df.query("starters != '0'")

    winners = []
    losers = []
    scores = {}
    games = {}

    # Go throughe ach matchup for the week
    for matchup in df['matchup_id'].unique():        
        
        # Limit the data to just the current matchup
        matchup_df = df.loc[df['matchup_id']==matchup]

        # Compare the two players in the matchup
        a = [matchup_df['roster_id'].unique()[0], matchup_df['points'].unique()[0]]
        b = [matchup_df['roster_id'].unique()[1], matchup_df['points'].unique()[1]]

        scores[a[0]] = a[1]
        scores[b[0]] = b[1]

        games[matchup] = a[1]+b[1]

        # Format strings
        if a[1] > b[1]:
            w = "{} (vs {})".format(a[0], b[0])
            l = "{} (vs {})".format(b[0], a[0])
            winners.append(w)
            losers.append(l)
        elif a[1] < b[1]:
            w = "{} (vs {})".format(b[0], a[0])
            l = "{} (vs {})".format(a[0], b[0])        
            winners.append(w)
            losers.append(l)            
        else:
            w = "{} tied {}!".format(a[0], b[0])
            l = w
            winners.append(w)
            losers.append(l)            
    
    # Find top scorers, games, etc.
    top_scorer = max(scores.items(), key = lambda k : k[1])
    low_game = list(min(games.items(), key = lambda k : k[1]))
    top_game = list(max(games.items(), key = lambda k : k[1]))
    top_game[0] = list(df.loc[df['matchup_id']==top_game[0]]['roster_id'].unique())
    low_game[0] = list(df.loc[df['matchup_id']==low_game[0]]['roster_id'].unique())

    # Add in positions and player names to dataframe
    df['Position'] = [players_df.loc[players_df['ID']==id]['Position'].unique()[0] for id in df['starters']]
    df['Player Name'] = [players_df.loc[players_df['ID']==id]['Name'].unique()[0] for id in df['starters']]   

    # Get the highest scorers by position
    highest_df = df[['Position', 'starters_points', 'Player Name', 'roster_id']].sort_values("starters_points", ascending = False).groupby("Position", as_index=False).first()
    highest_df.columns = ['Position', 'Points', 'Name', 'Team']

    # Return relevant data
    return df, "WINNERS: {}".format(', '.join(winners)), "LOSERS: {}".format(', '.join(losers)), "Highest Scorer: {}".format(list(top_scorer)), \
        "Highest Scoring Game: {}".format(list(top_game)), "Lowest Scoring Game (Boo!): {}".format(list(low_game)), highest_df

# Function to compare the points of two teams
def compare_points(df, players_df, requirements, year, week):

    # Calculate weekly scores for starters
    starters = [str(x) for x in df['starters']]
    starters_names = [players_df.loc[players_df['ID']==id]['Name'].to_list()[0] for id in starters]
    starters_positions = [players_df.loc[players_df['ID']==id]['Position'].to_list()[0] for id in starters]
    starters_df = pd.DataFrame({'Player':starters_names, 'Position': starters_positions, 'Score': get_stats(starters, year, week)})
    starters_df.fillna(0, inplace=True)
    starters_df['Score'] = [x['pts_half_ppr'] if x != 0 else 0 for x in starters_df['Score']]
    starters_df.fillna(0, inplace=True)
    starters_df['Type'] = 'Starter'

    # Get total starter points
    starter_points = starters_df['Score'].sum()

    # Calculate scores for all roster spots
    _all = [x for x in df['players'].to_list()[0]]
    all_names = [players_df.loc[players_df['ID']==id]['Name'].to_list()[0] for id in _all]
    all_positions = [players_df.loc[players_df['ID']==id]['Position'].to_list()[0] for id in _all]
    all_df = pd.DataFrame({'Player':all_names, 'Position': all_positions, 'Score': get_stats(_all, year, week)})
    all_df.fillna(0, inplace=True)
    all_df['Score'] = [x['pts_half_ppr'] if x != 0 else 0 for x in all_df['Score']]
    all_df.fillna(0, inplace=True)

    # Organize by top scorers by position
    all_points = all_df.sort_values("Score", ascending = False).groupby("Position", as_index=False).head(n=1000)

    # Use the requirements dictionary of roster makeup to calculate the optimal roster based on points scored
    FLEX = requirements['FLEX']
    select_players = lambda x: x.nlargest(requirements[x.name])

    idx_best = all_points.groupby('Position')['Score'].apply(select_players).index.levels[1]
    idx_flex = all_points.loc[all_points.index.difference(idx_best), 'Score'].nlargest(FLEX).index

    # out = all_points.loc[idx_best.union(idx_flex)].sort_values('Score', ascending=False)
    max_points = all_points.loc[idx_best.union(idx_flex)].sort_values('Score', ascending=False)['Score'].sum()

    # Return a list containing 0 == max possible points, and 1 == points scored
    return [float(max_points), float(starter_points)]         

# Get weekly matchup data
def get_weekly_data(league, week: int, lookup_dict):
    # Normalize the returned json string
    weekly_df = pd.json_normalize(league.get_matchups(week)).explode(['starters', 'starters_points'])
    # Pandas magic
    weekly_df = weekly_df[weekly_df.columns.drop(list(weekly_df.filter(regex="players_points.")))]
    weekly_df.drop('custom_points', inplace=True, axis=1)
    weekly_df['roster_id'] = weekly_df['roster_id'].map(lookup_dict)    

    # Return df of weekly matchups
    return weekly_df    

## Script

In [4]:
# Query sleeper API for list of all players
result_json_string = requests.get("https://api.sleeper.app/v1/players/nfl");

# Try to query
try:
    result_json_string.raise_for_status()
except requests.exceptions.HTTPError as e:
    print(e)

# Convert to json
players_json = result_json_string.json()

# Get the players dataframe
players_df = parse_players(players_json)

100%|██████████| 8094/8094 [00:00<00:00, 898470.20it/s]


In [5]:
# weekly_df = get_weekly_data(league, week, lookup_dict)
weekly_df = get_weekly_data(league, 10, lookup_dict)

In [6]:
df, winners, losers, scores, high, low, positions = get_league_data(weekly_df, players_df)

print_str = """

WINNERS: {} \n
LOSERS: {} \n
Highest Scorer: {} \n
Highest Scoring Game: {} \n
Lowest Scoring Game (Boo!): {} \n
Top Scoring Players by Position
-------------------------------------
{}


""".format(winners, losers, scores, high, low, positions)

print(print_str)



WINNERS: WINNERS: Robby (vs Tim), Matt (vs BJ), Tony (vs Chris), Erik (vs Kade), Ric (vs Joey) 

LOSERS: LOSERS: Tim (vs Robby), BJ (vs Matt), Chris (vs Tony), Kade (vs Erik), Joey (vs Ric) 

Highest Scorer: Highest Scorer: ['Ric', 126.82] 

Highest Scoring Game: Highest Scoring Game: [['Joey', 'Ric'], 242.95999999999998] 

Lowest Scoring Game (Boo!): Lowest Scoring Game (Boo!): [['Robby', 'Tim'], 191.72] 

Top Scoring Players by Position
-------------------------------------
  Position  Points             Name   Team
0      DEF   24.00   Dallas Cowboys    Tim
1        K   12.00    Chris Boswell  Chris
2       QB   36.24  Patrick Mahomes   Erik
3       RB   25.80        AJ Dillon   Erik
4       TE   19.90     Travis Kelce  Robby
5       WR   27.80     Deebo Samuel   Matt





In [31]:
def wrapper(league, week, players_df, requirements, lookup, records):
    
    points = {}

    weekly_data = get_weekly_data(league, week, lookup)
    league_df, winners, losers, scores, high, low, positions = get_league_data(weekly_data, players_df)

    for team in league_df['roster_id'].unique():
        temp = league_df.loc[league_df['roster_id']==team]
        res = compare_points(temp, players_df, requirements, 2021, week)
        points[team] = res
    

    # Process against median
    median_points = np.median([v[1] for k,v in points.items()])
    print("Median points for week {} was {}".format(week, median_points))
    for k,v in points.items():
        if v[1] > median_points:
            records.get(k)[0] += 1
        else:
            records.get(k)[1] += 1
    
    

    return records

In [49]:
records = {
    'Tony': [7, 3],
    'Robby': [10, 0],
    'Matt': [7, 3],
    'BJ': [2, 8],
    'Chris': [3, 7],
    'Tim': [4, 6],
    'Kade': [4, 6],
    'Joey': [5,5],
    'Erik': [6,4],
    'Ric': [2,8]
}

In [50]:
for i in range(0,10):
    week = i+1
    records = wrapper(league, week, players_df, requirements_dict, lookup_dict, records)

Median points for week 1 was 121.96000000000001
Median points for week 2 was 123.11000000000001
Median points for week 3 was 117.08000000000001
Median points for week 4 was 124.13999999999999


C:\Users\Kelly\AppData\Local\Temp/ipykernel_23992/3579991185.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Position'] = [players_df.loc[players_df['ID']==id]['Position'].unique()[0] for id in df['starters']]
C:\Users\Kelly\AppData\Local\Temp/ipykernel_23992/3579991185.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Player Name'] = [players_df.loc[players_df['ID']==id]['Name'].unique()[0] for id in df['starters']]


Median points for week 5 was 132.07
Median points for week 6 was 125.55999999999999
Median points for week 7 was 122.13999999999999
Median points for week 8 was 110.96000000000001
Median points for week 9 was 100.22999999999999
Median points for week 10 was 111.16999999999999


In [51]:
records

{'Tony': [15, 5],
 'Robby': [18, 2],
 'Matt': [15, 5],
 'BJ': [4, 16],
 'Chris': [6, 14],
 'Tim': [8, 12],
 'Kade': [8, 12],
 'Joey': [9, 11],
 'Erik': [12, 8],
 'Ric': [5, 15]}

In [77]:
def run_sim(runs=1000000):

    min_t = 100
    max_t = 150

    min_b = 98
    max_b = 144

    mm_t = MinMaxScaler(feature_range=(min_t, max_t))
    mm_b = MinMaxScaler(feature_range=(min_b, max_b))

    t_wins = 0
    b_wins = 0

    for _ in tqdm(range(runs)):
        res_t = np.random.normal(size=1000)
        res_b = np.random.normal(size=1000)

        res_t = mm_t.fit_transform(res_t.reshape(-1,1))
        res_b = mm_b.fit_transform(res_b.reshape(-1,1))

        count_t = 0
        count_b = 0

        for x,y in zip(res_t, res_b):
            if x > y:
                count_t += 1
            elif x < y:
                count_b += 1
            else:
                continue

        if count_t > count_b:
            #print("T beats B by a score of {} to {}".format(count_t, count_b))
            t_wins += 1
        else:
            #print("T loses to B by a score of {} to {}".format(count_t, count_b))
            b_wins += 1
    
    print("T: {} | B: {}".format(t_wins, b_wins))


In [78]:
run_sim(10000)

 54%|█████▍    | 541729/1000000 [10:27<08:44, 874.50it/s]